In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 反馈或问题？

如有任何反馈或疑问，请打开一个问题（https://github.com/googleapis/python-aiplatform/issues）。

Python的Vertex SDK：AutoML文本提取示例

要使用这个Jupyter笔记本，请将笔记本复制到安装了Tensorflow的Google Cloud笔记本实例中并打开它。您可以运行每个步骤或单元格，并查看其结果。要运行一个单元格，请使用Shift+Enter。Jupyter会自动显示每个单元格中最后一行的返回值。有关在Google Cloud笔记本中运行笔记本的更多信息，请参阅[Google Cloud笔记本指南](https://cloud.google.com/vertex-ai/docs/general/notebooks)。

这个笔记本演示了如何使用Vertex AI文本数据集创建AutoML文本提取模型，以及如何为在线预测提供模型。

注意：您可能会因为在测试这个SDK时涉及培训、预测、存储或使用其他GCP产品而产生费用。

### 安装 Python 的 Vertex SDK


SDK 安装后，内核将自动重新启动。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 输入您的项目和GCS存储桶

在下面的单元格中输入您的项目ID。然后运行单元格，确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

In [ ]:
MY_PROJECT = "YOUR PROJECT ID"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

初始化Python的Vertex SDK

为Vertex AI初始化*客户端*。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

## 在 Vertex AI 上创建一个数据集
我们现在将使用先前准备好的 jsonl 文件创建一个 Vertex AI 文本数据集。

### 生物医学数据集
为了创建一个实体抽取模型，使用一个包含数百种疾病和概念的生物医学研究摘要语料库。生成的模型可以在其他文档中识别这些医学实体。

该语料库的目标是通过基于文本的反思来推进对幸福原因的理解。

更多信息请参考[AutoML文档](https://cloud.google.com/natural-language/automl/docs/quickstart#model_objectives)。

In [ ]:
# Text Extraction
IMPORT_FILE = "gs://ucaip-test-us-central1/dataset/ucaip_ten_dataset.jsonl"

In [ ]:
ds = aiplatform.TextDataset.create(
    display_name="text-extraction",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.extraction,
)

ds.resource_name

在Vertex AI上启动一个训练工作，并创建一个模型

In [ ]:
job = aiplatform.AutoMLTextTrainingJob(
    display_name="text-extraction", prediction_type="extraction"
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    model_display_name="text-extraction",
)

部署模型

In [ ]:
endpoint = model.deploy()

# 在终端上预测

In [ ]:
input_text = """
Phenotypic variation including retinitis pigmentosa, pattern dystrophy, and fundus flavimaculatus in a single family with a deletion of codon 153 or 154 of the peripherin/RDS gene.\tBACKGROUND AND OBJECTIVES  Mutations of the peripherin / RDS gene have been reported in autosomal dominant retinitis pigmentosa , pattern macular dystrophy , and retinitis punctata albescens . We report herein the occurrence of three separate phenotypes within a single family with a novel 3-base pair deletion of codon 153 or 154 of the peripherin / RDS gene . DESIGN  Case reports with clinical features , fluorescein angiography , kinetic perimetry , electrophysiological studies , and molecular genetics . SETTING  University medical centers . PATIENTS  A 75-year-old woman , her two daughters ( aged 44 and 50 years ) , and her 49-year-old son were screened for peripherin / RDS mutations because of the presence of multiple phenotypes within the same family . RESULTS  The mother presented at age 63 years with a profoundly abnormal electroretinogram ( ERG ) and adult-onset retinitis pigmentosa that progressed dramatically over 12 years , with marked loss of peripheral visual field . One daughter developed pattern macular dystrophy at age 31 years . At age 44 years , her ERG was moderately abnormal but her clinical disease was limited to the macula . Another daughter presented at age 42 years with macular degeneration and over 10 years developed the clinical picture of fundus flavimaculatus . Her peripheral visual field was preserved but her ERG was moderately abnormal . The son had onset of macular degeneration at age 44 years . Pericentral scotomas were present and the ERG was markedly abnormal . Fluorescein angiography revealed punctate pigment epithelial transmission defects . CONCLUSIONS  A 3-base pair deletion of codon 153 or 154 of the peripherin / RDS gene can produce clinically disparate phenotypes even within the same family
Splicing defects in the ataxia-telangiectasia gene, ATM: underlying mutations and consequences.\tMutations resulting in defective splicing constitute a significant proportion ( 30 / 62 [ 48 % ] ) of a new series of mutations in the ATM gene in patients with ataxia-telangiectasia ( AT ) that were detected by the protein-truncation assay followed by sequence analysis of genomic DNA . Fewer than half of the splicing mutations involved the canonical AG splice-acceptor site or GT splice-donor site . A higher percentage of mutations occurred at less stringently conserved sites , including silent mutations of the last nucleotide of exons , mutations in nucleotides other than the conserved AG and GT in the consensus splice sites , and creation of splice-acceptor or splice-donor sites in either introns or exons . These splicing mutations led to a variety of consequences , including exon skipping and , to a lesser degree , intron retention , activation of cryptic splice sites , or creation of new splice sites . In addition , 5 of 12 nonsense mutations and 1 missense mutation were associated with deletion in the cDNA of the exons in which the mutations occurred . No ATM protein was detected by western blotting in any AT cell line in which splicing mutations were identified . Several cases of exon skipping in both normal controls and patients for whom no underlying defect could be found in genomic DNA were also observed , suggesting caution in the interpretation of exon deletions observed in ATM cDNA when there is no accompanying identification of genomic mutations .
"""

instances_list = [{"content": input_text}]

prediction = endpoint.predict(instances_list)
prediction

In [ ]:
prediction_instance = prediction.predictions[0]

extractions = zip(
    prediction_instance["ids"],
    prediction_instance["textSegmentStartOffsets"],
    prediction_instance["textSegmentEndOffsets"],
    prediction_instance["confidences"],
    prediction_instance["displayNames"],
)

for id, start, end, confidence, display_name in extractions:
    print(
        f"{id}: '{input_text[int(start):int(end)]}' predicted as '{display_name}'' with confidence {confidence}"
    )